## Global Seawater Data Processing
This notebook processes the temperature, salinity, dissolved oxygen and pH data downloaded from various online sources. 
The netCDF file format is commonly used for storing multidimensional scientific data (eg. temperature, salinity).

## **Temperature**

> EN4 is the UK Met Office Hadley Centre’s subsurface temperature and salinity dataset for the global oceans, spanning 1900 to present at a monthly timestep.

Each file is for a single month, therefore each file has only one time value

**Fields:**
- temperature 
- salinity

https://www.metoffice.gov.uk/hadobs/en4/download-en4-2-2.html

In [ ]:
import os
import xarray as xr

def extract_netcdf_en4(filename, date, param_OI):
    """
    Inputs:
        filename: str
        date: date to extract data for
        param_OI: parameter of interest (eg. temperature)
    """
    
    file_path = os.path.join(base_dir, filename)
    ds = xr.open_dataset(file_path, decode_times=False)
    data = ds.sel(depth=ds.depth[0], time=ds.time[date])  # pressure in decibars (2.5 is the closest depth to sea-level)

    # extract lat, lon and desired parameter data
    lat = data.lat
    lon = data.lon

    if param_OI=='temperature':
        param = data[param_OI] - 273.15
    else:
        param = data[param_OI]

    time = float(ds.time[date])
    date_str = convert_days_since(time, 1800, 1, 1)

    return lat, lon, param, date_str

In [ ]:
def format_temp(lat, lon, temp):
    lats = np.arange(-90, 90, 1)
    lons = np.arange(0, 361, 1)
    lon_shifted = lon - 180
    lon_wrapped = (lon_shifted + 180) % 360 - 180

    df_temp = pd.DataFrame(index=lats, columns=lon_wrapped)
    df_temp.loc[lat, lon_wrapped] = temp.values
    df_temp = df_temp.apply(pd.to_numeric)
    df_temp.columns = np.arange(len(df_temp.columns))
    df_temp = df_temp[np.concatenate((np.arange(180, 360), np.arange(0, 181)))]
    df_temp.columns = np.arange(-180,181,1)

    return df_temp

In [ ]:
def extract_en4_data(filename, date, param_OI):

    file_path = os.path.join(base_dir, filename)
    ds = xr.open_dataset(file_path, decode_times=False)

    # Convert time values from days since 1800-01-01 to normal date format
    time_values_days = ds['time'].values
    start_date = pd.Timestamp('1800-01-01')
    time_values_normal = [start_date + pd.Timedelta(days=float(days)) for days in time_values_days]

    date_values = [str(dt.date()) for dt in time_values_normal]

    # boolean mask for the desired date and filter
    mask_desired_date = [d.startswith(date) for d in date_values]
    data = ds.sel(depth=ds.depth[0], time=ds['time'][mask_desired_date])

    # extract lat, lon and desired parameter data
    lat = data.lat
    lon = data.lon

    if param_OI=='temperature':
        param = data[param_OI] - 273.15
    else:
        param = data[param_OI]

    return lat, lon, param

## **Salinity**
This also uses the EN4 dataset

In [ ]:
def format_sal(lat, lon, sal):
    lats = np.arange(-90, 90, 1)
    lons = np.arange(-180, 181, 1)

    lon = np.where(lon > 180, lon - 360, lon)

    lon_grid, lat_grid = np.meshgrid(lons, lats)
    df_sal = pd.DataFrame(index=lats, columns=lons)
    df_sal.loc[lat, lon] = sal.values
    df_sal = df_sal.apply(pd.to_numeric)

    return df_sal

## **Dissolved Oxygen**
> GOBAI-O2 is a gridded dataset on dissolved oxygen ranging from January 2004 to December 2022 at monthly resolution

time units: days since 2004-01-01

**Fields:**
- dissolved oxygen


https://catalog.data.gov/dataset/gobai-o2-a-global-gridded-monthly-dataset-of-ocean-interior-dissolved-oxygen-concentrations-bas

In [ ]:
def extract_GOBAI_O2_Data(filename, date):
    """
    extracts dissolved oxygen data from the GOBAI-O2 database
    ---
    Inputs:
        filename: name of the netcdf file
        month_idx: index of the month of interest within the dataset

    NB:
        the time field has units: days since 2004-01-01
    """

    file_path = os.path.join(base_dir, filename)
    ds = xr.open_dataset(file_path, decode_times=False)

    # Convert time values from days since 2004-01-01 to normal date format
    time_values_days = ds['time'].values
    start_date = pd.Timestamp('2004-01-01')
    time_values_normal = [start_date + pd.Timedelta(days=float(days)) for days in time_values_days]

    date_values = [str(dt.date()) for dt in time_values_normal]

    # boolean mask for the desired date and filter
    mask_desired_date = [d.startswith(date) for d in date_values]
    data = ds.sel(time=ds['time'][mask_desired_date], pres=2.5) # pressure in decibars (2.5 is the closest depth to sea-level)

    # extract parameter value arrays
    lat = data.lat.values
    lon = data.lon.values
    doxy = data["oxy"].values

    # sort latitude and longitude arrays for plotting
    lat_sorted_idx = np.argsort(lat)
    lon_sorted_idx = np.argsort(lon)
    lat_sorted = lat[lat_sorted_idx]
    lon_sorted = lon[lon_sorted_idx]
    lon_sorted_idx_2d, lat_sorted_idx_2d = np.meshgrid(lon_sorted_idx, lat_sorted_idx)
    doxy_sorted = doxy[0, lat_sorted_idx_2d, lon_sorted_idx_2d]

    return lat_sorted, lon_sorted, doxy_sorted


In [ ]:
def format_doxy(lat, lon, doxy):
    whole_lats = np.arange(-90, 90)
    whole_lons = np.arange(-180, 181)

    lon_1d, lat_1d = np.meshgrid(lon, lat)
    lon_flat = lon_1d.flatten()
    lat_flat = lat_1d.flatten()

    whole_lon_grid, whole_lat_grid = np.meshgrid(whole_lons, whole_lats)
    doxy_whole_grid = griddata((lon_flat, lat_flat), doxy.flatten(), (whole_lon_grid, whole_lat_grid), method='linear')

    df_doxy = pd.DataFrame(data=doxy_whole_grid, index=whole_lats, columns=whole_lons)

    return df_doxy

## **pH**
> OceanSODA-ETHZ is a global gridded data set of the surface ocean carbonate system for seasonal to decadal studies of ocean acidification (ETH Zürich)

This dataset extrapolates in time and space the surface ocean observations of pCO2 (from the Surface Ocean CO2 ATlas (SOCAT)) and total alkalinity (TA, from the Global Ocean Data Analysis Project (GLODAP)).

**Fields:**
- pH

not used:
- salinity 
- temperature 

In [ ]:
def extract_netcdf_dt(filename, date, param_OI):
    """
    reads netcdf file and extracts data for a specific date
    ___
    Inputs:
        filename: name of the netcdf file
        date: date of interest to extract data for
        param_OI: parameter of interest to extract data for
            - ph_total (for pH from OceanSODA-ETHZ)

    Outputs:
        lat: 1° grid of latitude coordinates
        lon: 1° grid of longitude coordinates
        param: lat x lon array of parameter value for each grid cell
    """
    current_dir = os.getcwd()  # Get the current working directory
    file_path = os.path.join(current_dir, "data", filename)
    ds = xr.open_dataset(file_path)
    data = ds.sel(time=date)

    # extract lat, lon and desired parameter data
    lat = data.lat
    lon = data.lon
    param = data[param_OI]

    return lat, lon, param

In [ ]:
def format_pH(lat, lon, pH):

    whole_lats = np.arange(-90, 90)
    whole_lons = np.arange(-180, 181)

    lon_1d, lat_1d = np.meshgrid(lon, lat)
    lon_flat = lon_1d.flatten()
    lat_flat = lat_1d.flatten()

    whole_lon_grid, whole_lat_grid = np.meshgrid(whole_lons, whole_lats)

    whole_lon_flat = whole_lon_grid.flatten()
    whole_lat_flat = whole_lat_grid.flatten()

    # Use the updated data to interpolate pH values onto the whole_lon_grid and whole_lat_grid
    pH_whole_grid = griddata((lon_flat, lat_flat), pH.values.flatten(), (whole_lon_flat, whole_lat_flat), method='linear')

    # Reshape the interpolated data to the original grid
    pH_whole_grid = pH_whole_grid.reshape(whole_lon_grid.shape)

    df_ph = pd.DataFrame(data=pH_whole_grid, index=whole_lats, columns=whole_lons)
    
    return df_ph

### Generating the Dataframes

The following two cells combine all the en4 zip files for a specific year (eg: EN.4.2.2.analyses.g10.2000.zip, EN.4.2.2.analyses.g10.2001.zip, etc) into a single netcdf file.

In [ ]:
import os
import requests


current_dir = os.getcwd()  # Get the current working directory
destination_folder = os.path.join(current_dir, "data")

def download_files(urls, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Download files that end with '2000' up to '2023'
    for url in urls:
        filename = os.path.basename(url)
        if filename.endswith(".zip") and filename.split(".")[-2].endswith(tuple(str(year) for year in range(2000, 2024))):
            file_path = os.path.join(destination_folder, filename)

            print(f"Downloading {filename}...")

            # Download the file
            response = requests.get(url)
            if response.status_code == 200:
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"{filename} downloaded successfully.")
            else:
                print(f"Failed to download {filename}. Status code: {response.status_code}")

    print("All files downloaded.")

download_files(urls, destination_folder)


import zipfile
extraction_folder = os.path.join(destination_folder, "en4")

def process_extracted_files(extraction_folder):
    for root, _, files in os.walk(extraction_folder):
        for file in files:
            file_path = os.path.join(root, file)
            print(f"Processing {file_path}")

# Extract files from the downloaded zip files
for root, _, files in os.walk(destination_folder):
    for file in files:
        if file.endswith(".zip"):
            zip_file_path = os.path.join(destination_folder, file)
            with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(extraction_folder)

process_extracted_files(extraction_folder)

In [ ]:
import os
import xarray as xr

def merge_nc_files(input_dir, output_file):
    # Get a list of all NetCDF files in the input directory
    nc_files = [f for f in os.listdir(input_dir) if f.endswith(".nc")]

    if not nc_files:
        print("No NetCDF files found in the directory.")
        return

    data_arrays = []

    for nc_file in nc_files:
        file_path = os.path.join(input_dir, nc_file)
        data = xr.open_dataset(file_path)
        data_arrays.append(data)

    merged_data = xr.concat(data_arrays, dim="time")

    merged_data.to_netcdf(output_file)

    print(f"Merged {len(nc_files)} NetCDF files into {output_file}.")


output_file = os.path.join(current_dir, "data", "en4_2000_2023.nc")
merge_nc_files(extraction_folder, output_file)


In [ ]:
def generate_dfs(date):
    # en4_2000_2023 is a netcdf file which combines all data from 2000-2023 in a single file (above)
    lat, lon, temp = extract_en4_data("datasets\en4_2000_2023.nc", date, "temperature")
    df_temp = format_temp(lat, lon, temp)

    lat, lon, sal = extract_en4_data("datasets\en4_2000_2023.nc", date, "salinity")
    df_sal = format_temp(lat, lon, sal)

    lat, lon, doxy = extract_GOBAI_O2_Data('datasets\GOBAI-O2-v2.1.nc', date)
    df_doxy = format_doxy(lat, lon, doxy)

    lat, lon, pH = extract_netcdf_dt(filename='datasets\OceanSODA_ETHZ-v2023.OCADS.01_1982-2022.nc', date=date+'-15', param_OI='ph_total')
    df_pH = format_pH(lat, lon, pH)

    return df_temp, df_sal, df_doxy, df_pH

In [ ]:
current_dir = os.getcwd()  # Get the current working directory
save_directory = os.path.join(current_dir, "data", "monthly_data")

for year in range(2004, 2022):
    for i in range(1, 13):
        month = f'{i:02}' 
        df_temp, df_sal, df_doxy, df_pH = generate_dfs(f"{year}-{month}")

        dataframes = {
            'df_temp': df_temp,
            'df_sal': df_sal,
            'df_doxy': df_doxy,
            'df_pH': df_pH
        }

        filename = os.path.join(save_directory, f"{year}-{month}.pickle")
        pickle.dump(dataframes, open(filename, "wb"))

In [ ]:
nested_dataframes = []

# Loop through each pickle file in the directory
for filename in os.listdir(save_directory):
    if filename.endswith(".pickle"):
        filepath = os.path.join(save_directory, filename)
        loaded_dataframes = pickle.load(open(filepath, "rb"))

        nested_dataframes.append({
            'df_temp': loaded_dataframes['df_temp'],
            'df_sal': loaded_dataframes['df_sal'],
            'df_doxy': loaded_dataframes['df_doxy'],
            'df_pH': loaded_dataframes['df_pH']
        })

df_nested = pd.DataFrame(nested_dataframes)

file_names = [os.path.splitext(file)[0] for file in os.listdir(save_directory) if file.endswith(".pickle")]
df_nested.index = file_names

pickle_filename = os.path.join(save_directory, "ocean_params.pickle")
with open(pickle_filename, "wb") as file:
    pickle.dump(df_nested, file)

with open(pickle_filename, "rb") as file:
    df = pickle.load(file)
